In [19]:
# import packages
import sys
import os
sys.path.append(os.path.abspath('.'))

from pages.A_Explore_Preprocess_Dataset import load_dataset

In [20]:
# load dataset
filepath = "mortgage_3000_onehot.csv"
df = load_dataset(filepath)
df.head()

2025-05-08 15:39:09.038 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:09.039 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


,Unnamed: 0,pages_viewed,time_on_site_sec,propensity_score,bounced,started_application,used_calculator,completed_application,user_type_New,user_type_Returning,...,device_type_Tablet,entry_page_Calculator,entry_page_Contact,entry_page_Home,entry_page_Learn,entry_page_Rates,geo_region_Midwest,geo_region_Northeast,geo_region_South,geo_region_West
0,0,3,260,0.28,0,1,1,1,1,0,...,0,0,0,0,0,1,0,1,0,0
1,1,1,740,0.93,1,1,0,1,0,1,...,0,0,0,0,1,0,0,1,0,0
2,2,10,516,0.20,0,0,0,1,1,0,...,0,0,1,0,0,0,0,0,1,0
3,3,5,458,0.75,1,0,1,0,1,0,...,0,0,1,0,0,0,1,0,0,0
4,4,5,755,0.65,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1


In [21]:
import numpy as np
import pandas as pd
from pages.B_Train_Model import split_dataset, LinearRegression
from pages.C_Test_Model import rmse, mae, r2, compute_eval_metrics

# split dataset
target = 'propensity_score'
features = [col for col in df.select_dtypes(include='number').columns if col != target]

X = df[features].values
Y = df[[target]].values

X_train, X_val, y_train, y_val = split_dataset(X, Y, number=30)

# train 
model = LinearRegression(learning_rate=0.01, num_iterations=200)
model.fit(X_train, y_train)

def test(model):
    # predict
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)

    # evaluate
    metrics = ['mean_absolute_error', 'root_mean_squared_error', 'r2_score']
    train_metrics = compute_eval_metrics(X_train, y_train, model, metrics)
    val_metrics = compute_eval_metrics(X_val, y_val, model, metrics)

    print("Train Metrics:")
    print(train_metrics)

    print("\nValidation Metrics:")
    print(val_metrics)

test(model)

2025-05-08 15:39:11.318 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:11.319 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:11.319 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:11.321 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:11.324 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:11.325 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:11.325 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:11.326 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Train Metrics:
{'mean_absolute_error': 0.25376739393787084, 'root_mean_squared_error': 0.2921705908162253, 'r2_score': 0.00591988776325314}

Validation Metrics:
{'mean_absolute_error': 0.25152548471012465, 'root_mean_squared_error': 0.29078815153043114, 'r2_score': -0.010938289155578484}


In [22]:
# grid search for best learning rate - on normal Linear Regression
from pages.B_Train_Model import LinearRegression

learning_rates = [0.0001, 0.001, 0.01, 0.1, 0.5]
best_rmse = float('inf')
best_lr = None
best_model = None

for lr in learning_rates:
    model = LinearRegression(learning_rate=lr, num_iterations=200)
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)
    current_rmse = rmse(y_val, y_val_pred)

    print(f"Learning Rate: {lr}, RMSE: {current_rmse:.4f}")

    if current_rmse < best_rmse:
        best_rmse = current_rmse
        best_lr = lr
        best_model = model

print(f"\nBest Learning Rate: {best_lr}, RMSE: {best_rmse:.4f}")

2025-05-08 15:39:24.986 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:24.988 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:24.990 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:24.991 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:24.994 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:24.994 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:24.995 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:24.996 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Learning Rate: 0.0001, RMSE: 0.5553


2025-05-08 15:39:26.672 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:26.673 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:26.674 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:26.675 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:26.678 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:26.679 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:26.680 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:26.681 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Learning Rate: 0.001, RMSE: 0.4371


2025-05-08 15:39:28.480 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:28.480 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:28.481 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:28.483 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:28.488 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:28.489 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:28.490 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:28.491 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Learning Rate: 0.01, RMSE: 0.2908


2025-05-08 15:39:30.066 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:30.068 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:30.069 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:30.070 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:30.072 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:30.073 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:30.074 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:30.075 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Learning Rate: 0.1, RMSE: 0.2910


2025-05-08 15:39:31.865 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:31.866 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:31.866 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:31.870 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:31.871 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:31.872 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:31.873 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:31.876 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Learning Rate: 0.5, RMSE: 154.0592

Best Learning Rate: 0.01, RMSE: 0.2908


In [23]:
# grid search to choose l1 and l2 penalty
from pages.B_Train_Model import LassoRegression

l1_penalties = [0.01, 0.1, 0.5, 1.0]
best_rmse = float('inf')
best_l1 = None
best_lasso_model = None

for l1 in l1_penalties:
    model = LassoRegression(learning_rate=best_lr, num_iterations=200, l1_penalty=l1)
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)
    current_rmse = rmse(y_val, y_val_pred)

    print(f"L1 Penalty: {l1}, RMSE: {current_rmse:.4f}")

    if current_rmse < best_rmse:
        best_rmse = current_rmse
        best_l1 = l1
        best_lasso_model = model

print(f"\nBest L1 Penalty: {best_l1}, RMSE: {best_rmse:.4f}")

2025-05-08 15:39:57.059 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:57.060 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:57.060 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:57.061 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:57.064 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:57.065 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:57.065 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:57.065 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

L1 Penalty: 0.01, RMSE: 0.2908


2025-05-08 15:39:58.944 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:58.945 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:58.946 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:58.947 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:58.949 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:58.950 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:58.950 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:39:58.951 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

L1 Penalty: 0.1, RMSE: 0.2908


2025-05-08 15:40:00.627 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:00.628 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:00.630 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:00.632 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:00.650 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:00.651 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:00.653 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:00.656 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

L1 Penalty: 0.5, RMSE: 0.2907


2025-05-08 15:40:02.470 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:02.471 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:02.473 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:02.474 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:02.478 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:02.480 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:02.481 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:02.481 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

L1 Penalty: 1.0, RMSE: 0.2907

Best L1 Penalty: 1.0, RMSE: 0.2907


In [24]:
from pages.B_Train_Model import RidgeRegression

l2_penalties = [0.01, 0.1, 0.5, 1.0]
best_rmse = float('inf')
best_l2 = None
best_ridge_model = None

for l2 in l2_penalties:
    model = RidgeRegression(learning_rate=best_lr, num_iterations=200, l2_penalty=l2)
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)
    current_rmse = rmse(y_val, y_val_pred)

    print(f"L2 Penalty: {l2}, RMSE: {current_rmse:.4f}")

    if current_rmse < best_rmse:
        best_rmse = current_rmse
        best_l2 = l2
        best_ridge_model = model

print(f"\nBest L2 Penalty: {best_l2}, RMSE: {best_rmse:.4f}")

2025-05-08 15:40:22.550 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:22.551 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:22.552 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:22.553 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:22.557 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:22.559 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:22.560 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:22.560 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

L2 Penalty: 0.01, RMSE: 0.2908


2025-05-08 15:40:24.439 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:24.439 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:24.442 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:24.443 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:24.443 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:24.444 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:24.447 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:24.448 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

L2 Penalty: 0.1, RMSE: 0.2908


2025-05-08 15:40:26.046 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:26.047 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:26.048 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:26.051 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:26.052 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:26.054 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:26.054 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:26.059 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

L2 Penalty: 0.5, RMSE: 0.2908


2025-05-08 15:40:28.104 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:28.105 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:28.106 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:28.106 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:28.109 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:28.110 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:28.111 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:28.112 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

L2 Penalty: 1.0, RMSE: 0.2908

Best L2 Penalty: 0.01, RMSE: 0.2908


In [25]:
from pages.B_Train_Model import LassoRegression

lasso_model = LassoRegression(learning_rate=0.01, num_iterations=200, l1_penalty=best_l1)
lasso_model.fit(X_train, y_train)

test(lasso_model)

2025-05-08 15:40:39.072 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:39.073 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:39.074 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:39.075 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:39.080 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:39.080 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:39.081 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:39.081 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Train Metrics:
{'mean_absolute_error': 0.25379701592179515, 'root_mean_squared_error': 0.29218071983712485, 'r2_score': 0.005850960683751794}

Validation Metrics:
{'mean_absolute_error': 0.2514341392381936, 'root_mean_squared_error': 0.29067597980260507, 'r2_score': -0.01015849931743329}


In [26]:
from pages.B_Train_Model import RidgeRegression

ridge_model = RidgeRegression(learning_rate=best_lr, num_iterations=200, l2_penalty=best_l2)
ridge_model.fit(X_train, y_train)

test(ridge_model)

2025-05-08 15:40:53.478 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:53.480 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:53.481 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:53.481 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:53.486 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:53.487 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:53.488 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 15:40:53.489 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Train Metrics:
{'mean_absolute_error': 0.25376736170538194, 'root_mean_squared_error': 0.2921705244010992, 'r2_score': 0.005920339704349353}

Validation Metrics:
{'mean_absolute_error': 0.2515255592108344, 'root_mean_squared_error': 0.29078817303829113, 'r2_score': -0.01093843870169886}


In [27]:
!pip install -q matplotlib

In [27]:
# plot loss curves
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(model.cost_history, label='Linear Regression', linewidth=2)
plt.plot(lasso_model.cost_history, label='Lasso Regression', linestyle='--')
plt.plot(ridge_model.cost_history, label='Ridge Regression', linestyle=':')
plt.title('Loss Curves Over Iterations')
plt.xlabel('Iteration')
plt.ylabel('Loss (RMSE)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# save plot as image
plt.savefig('loss_curves.png', dpi=300)

/tmp/ipykernel_463934/1225307238.py:14: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [31]:
np.array(ridge_model.cost_history)-np.array(lasso_model.cost_history)

array([ 0.00000000e+00,  0.00000000e+00, -5.19776763e-06, -1.02085657e-05,
       -1.50403291e-05, -1.96937496e-05, -2.41733539e-05, -2.84828753e-05,
       -3.26248207e-05, -3.66032630e-05, -4.04212783e-05, -4.40814271e-05,
       -4.75880047e-05, -5.09421550e-05, -5.41493481e-05, -5.72110118e-05,
       -6.01290325e-05, -6.29085404e-05, -6.55521772e-05, -6.80597092e-05,
       -7.04373943e-05, -7.26879434e-05, -7.48118593e-05, -7.68065456e-05,
       -7.86889693e-05, -8.04526534e-05, -8.21034226e-05, -8.36407641e-05,
       -8.50665560e-05, -8.63901551e-05, -8.76082232e-05, -8.87274715e-05,
       -8.97487097e-05, -9.06746492e-05, -9.15009701e-05, -9.22444176e-05,
       -9.29006968e-05, -9.34697594e-05, -9.39596593e-05, -9.43704574e-05,
       -9.47048141e-05, -9.49622744e-05, -9.51470986e-05, -9.52679626e-05,
       -9.53195876e-05, -9.53110059e-05, -9.52415658e-05, -9.51137697e-05,
       -9.49300882e-05, -9.46848652e-05, -9.43967429e-05, -9.40599423e-05,
       -9.36767829e-05, -

Hyperparameter Search (Learning Rate, L1 Penalty, L2 Penalty)

In [ ]:
# grid search for best learning rate - on normal Linear Regression
from pages.B_Train_Model import LinearRegression

learning_rates = [0.0001, 0.001, 0.01, 0.1, 0.5]
best_rmse = float('inf')
best_lr = None
best_model = None

for lr in learning_rates:
    model = LinearRegression(learning_rate=lr, num_iterations=200)
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)
    current_rmse = rmse(y_val, y_val_pred)

    print(f"Learning Rate: {lr}, RMSE: {current_rmse:.4f}")

    if current_rmse < best_rmse:
        best_rmse = current_rmse
        best_lr = lr
        best_model = model

print(f"\nBest Learning Rate: {best_lr}, RMSE: {best_rmse:.4f}")

2025-05-08 14:20:09.039 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.039 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.039 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.039 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.042 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.042 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.042 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.043 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Learning Rate: 0.0001, RMSE: 0.5603


2025-05-08 14:20:09.568 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.569 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.571 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.572 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.577 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.578 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.578 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.578 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Learning Rate: 0.001, RMSE: 0.4926


2025-05-08 14:20:09.900 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.900 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.900 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.901 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.903 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.903 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.903 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:09.903 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Learning Rate: 0.01, RMSE: 0.3000


2025-05-08 14:20:10.206 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:10.206 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:10.206 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:10.206 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:10.209 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:10.210 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:10.210 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:10.210 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Learning Rate: 0.1, RMSE: 0.2973


2025-05-08 14:20:10.518 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:10.519 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:10.519 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:10.519 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:10.521 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:10.521 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:10.522 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:10.522 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Learning Rate: 0.5, RMSE: 0.2987

Best Learning Rate: 0.1, RMSE: 0.2973


In [ ]:
# grid search to choose l1 and l2 penalty
from pages.B_Train_Model import LassoRegression

l1_penalties = [0.01, 0.1, 0.5, 1.0]
best_rmse = float('inf')
best_l1 = None
best_lasso_model = None

for l1 in l1_penalties:
    model = LassoRegression(learning_rate=best_lr, num_iterations=200, l1_penalty=l1)
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)
    current_rmse = rmse(y_val, y_val_pred)

    print(f"L1 Penalty: {l1}, RMSE: {current_rmse:.4f}")

    if current_rmse < best_rmse:
        best_rmse = current_rmse
        best_l1 = l1
        best_lasso_model = model

print(f"\nBest L1 Penalty: {best_l1}, RMSE: {best_rmse:.4f}")

2025-05-08 14:20:06.303 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:06.304 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:06.304 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:06.304 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:06.307 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:06.307 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:06.307 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:06.308 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

L1 Penalty: 0.01, RMSE: 0.2973


2025-05-08 14:20:06.793 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:06.794 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:06.794 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:06.794 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:06.796 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:06.796 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:06.796 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:06.796 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

L1 Penalty: 0.1, RMSE: 0.2973


2025-05-08 14:20:07.094 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.094 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.094 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.094 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.096 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.096 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.097 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.097 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

L1 Penalty: 0.5, RMSE: 0.2972


2025-05-08 14:20:07.428 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.429 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.429 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.430 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.432 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.436 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.437 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.438 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

L1 Penalty: 1.0, RMSE: 0.2972

Best L1 Penalty: 1.0, RMSE: 0.2972


In [ ]:
from pages.B_Train_Model import RidgeRegression

l2_penalties = [0.01, 0.1, 0.5, 1.0]
best_rmse = float('inf')
best_l2 = None
best_ridge_model = None

for l2 in l2_penalties:
    model = RidgeRegression(learning_rate=best_lr, num_iterations=200, l2_penalty=l2)
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)
    current_rmse = rmse(y_val, y_val_pred)

    print(f"L2 Penalty: {l2}, RMSE: {current_rmse:.4f}")

    if current_rmse < best_rmse:
        best_rmse = current_rmse
        best_l2 = l2
        best_ridge_model = model

print(f"\nBest L2 Penalty: {best_l2}, RMSE: {best_rmse:.4f}")

2025-05-08 14:20:07.517 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.518 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.518 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.519 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.520 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.520 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.520 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:07.521 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

L2 Penalty: 0.01, RMSE: 0.2973


2025-05-08 14:20:08.012 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.012 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.012 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.013 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.014 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.014 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.014 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.014 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

L2 Penalty: 0.1, RMSE: 0.2973


2025-05-08 14:20:08.325 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.325 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.327 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.327 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.327 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.328 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.330 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.330 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

L2 Penalty: 0.5, RMSE: 0.2973


2025-05-08 14:20:08.581 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.581 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.581 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.581 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.583 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.583 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.584 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-08 14:20:08.584 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

L2 Penalty: 1.0, RMSE: 0.2973

Best L2 Penalty: 0.01, RMSE: 0.2973
